In [1]:
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime, timedelta
import pandas as pd
import os

In [2]:
fecha_hoy_format: str = "2024.06.21"

ruta_dacxa: str = "C:/Users/miria/Desktop/Disco Z Claro/JEFATURA CCD/Nuevo_DACxANALISTA.xlsx"

ruta_ipe_inicio: str = "C:/Users/miria/Desktop/Disco Z Claro/SALDOS A FAVOR//IPE/INICIO/IP_INICIO_2024.06.21.xlsx"

ruta_matriz: str = "C:/Users/miria/Desktop/Disco Z Claro/SALDOS A FAVOR/Saldos a Favor - Matriz/Saldos_a_Favor_Matriz_2024.06.21.xlsx"

In [3]:
columnas_originales = ["DEUDOR", "NOMBRE", "ESTADO", "REGION", "TIPO_DAC", "ANALISTA_ACT"]
columnas_renombradas = ["Cuenta", "Cliente", "Status", "Region", "Tipo", "Responsable"]

df_dacxa: pd.DataFrame = pd.read_excel(ruta_dacxa, sheet_name="Base_NUEVA", usecols=columnas_originales)
df_dacxa.rename(columns=dict(zip(columnas_originales, columnas_renombradas)), inplace=True)
df_dacxa = df_dacxa[columnas_renombradas]
df_dacxa.shape


(3913, 6)

In [4]:
df_dacxa = df_dacxa[(df_dacxa["Status"] != "LIQUIDADO") & (df_dacxa["Responsable"] != "SIN INFORMACION")]
df_dacxa.dropna(subset=["Cuenta"], inplace=True)
df_dacxa["Cuenta"] = df_dacxa["Cuenta"].astype("int64").astype(str)
df_dacxa.reset_index(drop=True, inplace=True)
df_dacxa.shape

(1258, 6)

In [5]:
df_dacxa.head(5)

,Cuenta,Cliente,Status,Region,Tipo,Responsable
0,4070459,CONTACTE S.A.,OPERATIVO SIN MOVIMIENTO,LIMA,AGENTE TELMEX,DIEGO RODRIGUEZ
1,4070467,CSD GLOBAL SERVICES S.A.C.,OPERATIVO SIN MOVIMIENTO,LIMA,AGENTE TELMEX,JOSE LUIS VALVERDE
2,4070600,SATELITAL TELECOMUNICACIONES S.A.C.,OPERATIVO CON MOVIMIENTO,LIMA,AGENTE TELMEX / RED,JUAN CARLOS HUATAY
3,4070168,HURTADO ARBILDO USER KIT,OPERATIVO SIN MOVIMIENTO,CENTRO,CDR,DIEGO RODRIGUEZ
4,4070192,SANCHEZ VILCA DE TAPARA VICTORIA,OPERATIVO SIN MOVIMIENTO,SUR,CDR,REGION SUR


In [6]:
columnas_inicio = ["Cuenta", "ACC", "Nº doc.", "Referencia", "Fecha doc.", "Fecha pago", "Clase", "Demora", "Mon.", "Importe en ML", "Texto"]

df_inicio: pd.DataFrame = pd.read_excel(ruta_ipe_inicio, usecols=columnas_inicio)
df_inicio = df_inicio[columnas_inicio]
df_inicio.shape

(21579, 11)

In [7]:
df_inicio["Cuenta"].isna().sum()

0

In [8]:
df_inicio["Cuenta"] = df_inicio["Cuenta"].astype("int64").astype(str)
df_inicio.dropna(subset=["Cuenta"], inplace=True)

df_inicio.shape

(21579, 11)

In [9]:
df_inicio = df_inicio[df_inicio["Importe en ML"] < 0]
df_inicio.reset_index(drop=True, inplace=True)
df_inicio.shape

(2031, 11)

In [10]:
df_inicio.head(5)

,Cuenta,ACC,Nº doc.,Referencia,Fecha doc.,Fecha pago,Clase,Demora,Mon.,Importe en ML,Texto
0,4070016,NaN,1420073721,NaN,2010-02-17,2010-02-18,C2,5237,PEN,-3000.0,DZ
1,4070016,NaN,1420052548,NaN,2012-01-15,2012-01-17,C2,4539,PEN,-1000.0,DZ
2,4070016,NaN,1420002342,NaN,2009-12-29,2009-12-30,C2,5287,PEN,-3572.0,DZ
3,4070088,NaN,1420060257,COMPENSACION,2010-11-30,2010-11-25,C2,4957,PEN,-30.0,DA compensacion
4,4070109,NaN,1420057093,SALDO A FAVOR,2011-11-30,2011-10-25,C2,4623,PEN,-68.0,DA COMPENSACIÓN


In [11]:
df_regla: pd.DataFrame = pd.read_excel("Regla.xlsx")
df_regla.shape

(15, 2)

In [12]:
df_regla.head(5)

,ACC,Producto
0,PE01,Pack Postpago
1,PE02,Pack Prepago
2,PE03,Recarga Virtual
3,PE04,Especial
4,PE05,TRANSACCIONES


In [13]:
df_inicio = pd.merge(df_inicio, df_regla, how="left", left_on="ACC", right_on="ACC")
df_inicio.shape

(2031, 12)

In [14]:
df_inicio.tail(5)

,Cuenta,ACC,Nº doc.,Referencia,Fecha doc.,Fecha pago,Clase,Demora,Mon.,Importe en ML,Texto,Producto
2026,4073842,PE17,4510069630,1704073842,2024-06-01,2024-06-01,J9,20,PEN,-38.56,NaN,HFC
2027,4073855,PE17,2290621251,1704073855,2024-06-06,2024-06-05,AB,16,PEN,-0.28,COMP AUTOM – CCD,HFC
2028,4073900,PE17,2290615208,1704073900,2024-06-04,2024-06-03,AB,18,PEN,-0.21,COMP AUTOM – CCD,HFC
2029,4074807,PE17,2290230160,1704074807,2024-03-03,2024-03-02,AB,111,PEN,-0.30,COMP AUTOM – CCD,HFC
2030,4074883,PE17,2290615330,1704074883,2024-06-04,2024-06-03,AB,18,PEN,-0.88,COMP AUTOM – CCD,HFC


In [15]:
df_inicio["Producto"].isna().sum()

138

In [16]:
df_matriz = pd.merge(df_inicio, df_dacxa, how="left", left_on="Cuenta", right_on="Cuenta")
columnas_finales = ["Cuenta", "ACC", "Producto", "Cliente", "Status", "Region", "Tipo", "Responsable", "Nº doc.", "Referencia", "Fecha doc.", "Fecha pago", "Clase", "Demora", "Mon.", "Importe en ML", "Texto"]
df_matriz = df_matriz[columnas_finales]
df_matriz.shape

(2031, 17)

In [17]:
df_matriz.dropna(subset=["Cuenta", "Status"], inplace=True)
df_matriz.shape

(1546, 17)

In [18]:
# fillna "ACC" y "Producto" con 0
df_matriz["ACC"].fillna("0", inplace=True)
df_matriz["Producto"].fillna("0", inplace=True)
df_matriz.head(5)

,Cuenta,ACC,Producto,Cliente,Status,Region,Tipo,Responsable,Nº doc.,Referencia,Fecha doc.,Fecha pago,Clase,Demora,Mon.,Importe en ML,Texto
6,4070143,0,0,ECOMDATA PERU S.A.C.,OPERATIVO CON MOVIMIENTO,LIMA,INTEGRADOR/TLV CONTACT CENTER,JOSE LUIS VALVERDE,4331237497,E1-0140-00004620,2024-06-11,2024-06-11,J3,10,PEN,-129.00,DETRACCION-E1-0140-00004620
8,4070218,0,0,AVELLANEDA MEDINA JUDITH MARIBEL,PROCESO DE LIQUIDACIÓN,CENTRO,PVA,WALTER LOPEZ,1420171759,SALDO A FAVOR,2014-08-01,2014-07-31,C2,3613,PEN,-12.99,DA COMPENSACION
17,4070624,0,0,TAQUIMA CUTI HILARIO,OPERATIVO SIN MOVIMIENTO,SUR,PVA,REGION SUR,1420109612,0910015546,2013-10-31,2013-10-29,C2,3888,PEN,-38.00,DA COMPENSACION
48,4070792,0,0,WORLD CONNECT E.I.R.L.,OPERATIVO CON MOVIMIENTO,SUR,DACE,REGION SUR,2290600794,SALDO DE COMPENS,2024-05-31,2024-05-28,AB,24,PEN,-1.63,SALDO DE COMPENS
52,4071001,0,0,CUEVA CHUN PEDRO MARCIANO,OPERATIVO SIN MOVIMIENTO,NORTE,PVA,REGION NORTE,1420132809,SALDO A FAVOR,2013-07-31,2013-07-02,C2,4007,PEN,-60.00,DA saldo a favor


In [43]:
sort_cols = ["Responsable", "Nro DAC's", "Importe en ML", "%"]
df_resumen = df_matriz.groupby(["Responsable"])["Importe en ML"].sum().reset_index()
df_resumen["Nro DAC's"] = df_matriz.groupby(["Responsable"])["Cliente"].nunique().values
df_resumen["%"] = (df_resumen["Importe en ML"] / df_resumen["Importe en ML"].sum()).round(2)
df_resumen = df_resumen.sort_values(by="Importe en ML", ascending=True)
df_resumen = df_resumen[sort_cols]
df_resumen.reset_index(drop=True, inplace=True)
df_resumen.loc[df_resumen.shape[0]] = ["Total General", df_resumen["Nro DAC's"].sum(), df_resumen["Importe en ML"].sum(), 1]
df_resumen

,Responsable,Nro DAC's,Importe en ML,%
0,REGION SUR,90,-1132220.02,0.29
1,REGION NORTE,149,-927238.47,0.24
2,YOLANDA OLIVA,37,-658727.73,0.17
3,JOSE LUIS VALVERDE,48,-491404.29,0.13
4,RAQUEL CAYETANO,31,-268189.92,0.07
5,DIEGO RODRIGUEZ,37,-221183.49,0.06
6,JUAN CARLOS HUATAY,31,-160743.29,0.04
7,WALTER LOPEZ,8,-9732.44,0.00
8,Total General,431,-3869439.65,1.00


In [44]:
with pd.ExcelWriter(ruta_matriz, engine="openpyxl") as writer:
    df_resumen.to_excel(writer, sheet_name="Resumen", index=False)
    df_matriz.to_excel(writer, sheet_name="Detalle", index=False)

In [45]:
os.startfile(ruta_matriz)